# Change Detection - Image Ratio Demo
This notebook detects change between satellite images taken at subsequent times over the same location.

The methods are calibrated on [Sentinel-2](https://sentinel.esa.int/web/sentinel/user-guides/sentinel-2-msi/overview) pre-post disaster imagery from the Caribbean and [Copernicus Emergency Mapping Service (EMS)](https://emergency.copernicus.eu/mapping/map-of-activations-rapid#zoom=3&lat=29.18235&lon=-70.57787&layers=BT00) ground truth damage assessment data. Section 4 allows you to evaluate predicted damages against reported damages for locations covered by EMS. The models were built using the [Descartes Labs](https://www.descarteslabs.com/) platform. If you wish to re-train the models or have more flexibility in your parameters, plots or methods, please visit the notebooks for each method at [thresholding.ipynb](./thresholding.ipynb) and [unet_classifier.ipynb](./unet_classifier.ipynb).
### Contents
- 1 - [Visualise Subsequent Imagery](#visualiseImagery)
- 2 - [Detect Change - Thresholding Method](#thresholding)
- 3 - [Detect Change - Trained Classifier Method](#classifier)
- 4 - [Evaluate Methods Against Ground Data](#evaluate)
____________
## Initialisation - Choose Location
Firstly we need to choose a location for which to detect change. You may either choose one of the locations with pre-assigned variables from the list displayed when running the first cell, or you can choose a new location by selecting 'New Location' which will then prompt you to provide variables of your own when running the second cell. Pre-set locations are the following:
- Training Set: Roseau, Dominica (pre-post hurricane Maria); Abricots, Haiti (pre-post hurricane Matthew)
- Test Set: Jeremie, Haiti (pre-post hurricane Matthew); Port Salut, Haiti (pre-post hurricane Matthew)
- Informal Settlements: Bidonville Killick Stenio Vincent, Port-au-Prince, Haiti (2018 to 2019); Parry Town, Ocho Rios, Jamaica (2018 to 2019)
- High Resolution: Hidalgo County, Texas, USA (2016 to 2019)

In [1]:
import demoFunctions as dfn # Import functions from demoFunction.py
location = dfn.chooseLocation() # Generate pre-defined location list
location # Display location dropdown

Dropdown(description='Location:', options=(('train - Roseau, Dominica', 0), ('train - Abricots, Haiti', 1), ('…

In [2]:
# Assign variables for chosen location
variables = dfn.assignVariables(location) 

Button(description='Show variables', style=ButtonStyle())

Output()

In [3]:
# Submits input variables for new location (if not new location - no action)
variables = dfn.submitNewLocation(variables)

_____________
<a id='visualiseImagery'></a>
## 1 - Visualise Imagery
We'll start by displaying before and after images for the chosen location. Two pointers here:
- You'll need to click the magic markers below the map to scale the imagery band colours properly. You can also untick the 'After' box to toggle between pre and post disaster.
- If you have defined a new location and no imagery appears for your location, try increasing the cloud fraction, but check this does not lead to overly cloudy images which may affect detection performance. Alternatively try changing or widening the requested dates, it is not uncommon to span several months for a good image.

In [4]:
# Create map with before and after images for specified location
dfn.beforeAfterImages(variables)


`ipyleaflet` and/or `ipywidgets` Jupyter extensions are not installed! (or you're not in a Jupyter notebook.)
To install for JupyterLab, run this in a cell:
    !jupyter labextension install jupyter-leaflet @jupyter-widgets/jupyterlab-manager
To install for plain Jupyter Notebook, run this in a cell:
    !jupyter nbextension enable --py --sys-prefix ipyleaflet
Then, restart the kernel and refresh the webpage.


________
<a id='thresholding'></a>
## 2 - Detect Change - Ratio Thresholding
Next, let's take the logarithmic ratio of images for selected bands and display detected change superimposed on the image for time 2. Beneath the plot is a slider allowing you to vary the thresholds within which we are detecting change. If damage assessment data is available for the location this will be overlayed for a qualitative comparison.

> The optimal threshold interval for detecting building change has been determined as 0.01-0.1. However, due to differences in lighting between repeat images this may need adjsuted according to location. As the ratio is logarithmic, if the second image is considerably darker than the first, the appropriate thresholding may even be negative.

Detection interval equation for RGB (red, green and blue bands) where, for example, r1 denotes pixel value for red band at time 1:  $ threshold < \log\left(\frac{r2}{r1} \times \frac{b2}{b1} \times \frac{g2}{g1}\right) < cap  $

In [5]:
# Function to detect change through thresholding for location
plotVars = dfn.thresholding(variables)

`ipyleaflet` and/or `ipywidgets` Jupyter extensions are not installed! (or you're not in a Jup…

FloatRangeSlider(value=(0.1, 0.3), description='Filter ratios', max=0.5, min=-0.5, step=0.01)

Run box below to display updated detection result


In [6]:
# Re-plot detected change according to slider values
dfn.plotChange(plotVars)

You may observe 2 main limitations with this method.
- Lack of small scale change detections due to 10x10m resolution of Sentinel Imagery. Try re-running the notebook selecting the high resolution location to see what is possible where 1x1m resolution is available.
- Sensitivity of thresholding to image lighting and changes in areas other than buildings, due to effect on ratio values. If threshold is unadjusted or mask is not applied to ocean this leads to false detections. To mitigate this limitation we trained a [classifier](#classifier) in the next section to recognise the evidence of damage buildings within the image ratio rather than relying on simple thresholding.

____________________
<a id='classifier'></a>
## 3 - Detect Change - Ratio Classifier
To avoid the drawbacks of thresholding we trained a Convolutional Neural Network (CNN) with a [U-Net](https://arxiv.org/abs/1505.04597) architecture to identify building change from the pixel ratios between before/after Sentinel-2 imagery. This model evaluates change per 'image tile' of which there will be many within your area of interest. Therefore, first let's draw a polygon over the desired area. Then, each corresponding tile will individually be fed in to the model for assessing change detection. Bare in mind the larger the area the longer it will take to run the model for that area.
> One could question the decision not to just increase tilesize. However not only does this method make the evaluation area more flexible, but also the model does not cater well for tile sizes larger than that for which it was trained due to the input layer structure.

In [6]:
m3, testPoly = dfn.drawPolygon(variables) # Get map upon which to draw polygon for assessment
m3 # Display map


`ipyleaflet` and/or `ipywidgets` Jupyter extensions are not installed! (or you're not in a Jupyter notebook.)
To install for JupyterLab, run this in a cell:
    !jupyter labextension install jupyter-leaflet @jupyter-widgets/jupyterlab-manager
To install for plain Jupyter Notebook, run this in a cell:
    !jupyter nbextension enable --py --sys-prefix ipyleaflet
Then, restart the kernel and refresh the webpage.


In [7]:
# Detect change for all tiles at least partly within polygon
dfn.classifyDamage(testPoly, variables, m3)

Longitude Columns:   0%|          | 0/3 [00:00<?, ?it/s]

Number of tiles requested: 6 . Approximately 48 seconds on 16GB RAM.

Job ID: 24fffee13c1861da59e46fc6e72920ea7ce9949cadd381d2
[######] | Steps: 373/373 | Stage: SUCCEEDED                                  


Longitude Columns:  33%|███▎      | 1/3 [00:22<00:45, 22.99s/it]


Job ID: a993238645bac138f3a5ed8fdc4264cd11798b173a31c136
[######] | Steps: 373/373 | Stage: SUCCEEDED                                  


Longitude Columns:  67%|██████▋   | 2/3 [00:44<00:22, 22.68s/it]

...


Longitude Columns:  33%|███▎      | 1/3 [00:25<00:50, 25.02s/it]


Job ID: b4f61bd3bfd627e01a971f4c55e278ac8b6630c4fca2aad9
[######] | Steps: 373/373 | Stage: SUCCEEDED                                  WARNING:tensorflow:5 out of the last 5 calls to <function Model.make_predict_function.<locals>.predict_function at 0x7f26ed514cb0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for  more details.



Longitude Columns:  67%|██████▋   | 2/3 [00:44<00:23, 23.34s/it]


Job ID: 4d8ea30ff4086acc5d610d179e5f0820ec37af5c0b71ddb9
[######] | Steps: 373/373 | Stage: SUCCEEDED                                  WARNING:tensorflow:6 out of the last 6 calls to <function Model.make_predict_function.<locals>.predict_function at 0x7f26ec577560> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for  more details.



Latitude Rows: 100%|██████████| 2/2 [02:33<00:00, 76.88s/it]


# --------------- END ------------------------